In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#format for floats
pd.options.display.float_format = '{:,.8f}'.format

In [51]:
df0 = pd.read_csv('../Neueda2023/Datasets/2010.csv')
df1 = pd.read_csv('../Neueda2023/Datasets/2011.csv')
df2 = pd.read_csv('../Neueda2023/Datasets/2012.csv')
df3 = pd.read_csv('../Neueda2023/Datasets/2013.csv')
df4 = pd.read_csv('../Neueda2023/Datasets/2014.csv')
df5 = pd.read_csv('../Neueda2023/Datasets/2015.csv')
df6 = pd.read_csv('../Neueda2023/Datasets/2016.csv')

In [52]:
df_combined = pd.concat([df0, df1, df2, df3, df4, df5, df6])

In [53]:
!pip install pyxlsb

In [85]:
df_ev = pd.read_excel("./Datasets/vehicle.xlsb", sheet_name="County", engine="pyxlsb", header=1)
df_ev

,state_abbr,geoid,county_id,county_name,fuel_type_org,fuel_type,class,before 1980,1980-99,1990-99,...,2009.1,2010.1,2011.1,2012.1,2013.1,2014.1,2015.1,2016.1,2017.1,2018.1
0,AL,1001,1001,Autauga,BI,Other/Unknown,Car,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,1001,1001,Autauga,BI,Other/Unknown,Truck,NaN,NaN,0.00001904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL,1001,1001,Autauga,DIES,Diesel vehicle,Car,NaN,0.00019045,0.00003809,...,0.00000571,0.00000000,0.00000950,0.00000728,0.00000867,0.00001158,0.00000502,NaN,NaN,NaN
3,AL,1001,1001,Autauga,DIES,Diesel vehicle,Truck,NaN,0.00114268,0.00544679,...,0.00000000,0.00000000,0.00005713,0.00005112,0.00000000,0.00002943,0.00007849,0.00007428,0.00000000,NaN
4,AL,1001,1001,Autauga,DIES,Diesel vehicle,Unknown,0.00020949,0.00001904,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50810,WY,56045,56045,Weston,GAS,Gasoline vehicle,Unknown,0.04801597,0.00316976,0.00093919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50811,WY,56045,56045,Weston,HYBRID ELECTRIC VEHICLE,Hybrid electric vehicle,Car,NaN,NaN,NaN,...,0.00000000,0.00000000,0.00000000,0.00000267,0.00001526,0.00000787,0.00000286,0.00000000,0.00000000,NaN
50812,WY,56045,56045,Weston,HYBRID ELECTRIC VEHICLE,Hybrid electric vehicle,Truck,NaN,NaN,NaN,...,NaN,NaN,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50813,WY,56045,56045,Weston,PLUG IN HYBRID ELECTRIC VEHICLE,Plug-in hybrid electric vehicle,Car,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00000000,NaN,NaN,0.00000000,NaN,NaN


In [86]:
df_ev = df_ev.iloc[:, [0, 4, 42, 43, 44, 45, 46, 47, 48]]

In [116]:
df_ev = df_ev.fillna(0)

In [135]:
df_ev2 = df_ev.loc[df_ev['fuel_type_org'].isin(['ELECTRIC VEHICLE', 'HYBRID ELECTRIC VEHICLE', 'PLUG IN HYBRID ELECTRIC VEHICLE'])]
df_ev2

,state_abbr,fuel_type_org,2010.1,2011.1,2012.1,2013.1,2014.1,2015.1,2016.1
5,AL,ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000041,0.00000017,0.00000000
12,AL,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00001350,0.00002048,0.00001111,0.00001100,0.00000572
13,AL,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
14,AL,PLUG IN HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000090,0.00000000,0.00000000,0.00000051,0.00000000
24,AL,ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000029,0.00000028,0.00000043,0.00000050,0.00000000
...,...,...,...,...,...,...,...,...,...
50797,WY,PLUG IN HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000287,0.00000000,0.00000000,0.00000000,0.00000000
50804,WY,ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
50811,WY,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000267,0.00001526,0.00000787,0.00000286,0.00000000
50812,WY,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000


In [138]:
df_ev2 = df_ev2.groupby(['state_abbr']).sum()
df_ev2

,2010.1,2011.1,2012.1,2013.1,2014.1,2015.1,2016.1
state_abbr,,,,,,,
AK,0.00000000,0.00000000,0.00019404,0.00027505,0.00025085,0.00017206,0.00006107
AL,0.00000000,0.00015002,0.00085413,0.00092157,0.00069111,0.00064005,0.00041681
AR,0.00000000,0.00006496,0.00112876,0.00119191,0.00085840,0.00064756,0.00047690
AZ,0.00000000,0.00016319,0.00032641,0.00040572,0.00031892,0.00028002,0.00022568
CA,0.00000000,0.00256345,0.00309986,0.00460644,0.00386524,0.00424358,0.00328113
CO,0.00000000,0.00000000,0.00115534,0.00155333,0.00111488,0.00100616,0.00076280
CT,0.00000000,0.00009916,0.00026826,0.00031857,0.00027650,0.00020336,0.00020366
DC,0.00000000,0.00010189,0.00007690,0.00009504,0.00007977,0.00007863,0.00005972
DE,0.00000000,0.00000000,0.00010559,0.00014323,0.00010340,0.00009218,0.00008725


In [146]:
df_ev2 = df_ev2.unstack(level=0)

df_ev2

        state_abbr
2010.1  AK           0.00000000
        AL           0.00000000
        AR           0.00000000
        AZ           0.00000000
        CA           0.00000000
                        ...    
2016.1  VT           0.00060890
        WA           0.00097712
        WI           0.00095506
        WV           0.00041507
        WY           0.00015999
Length: 357, dtype: float64

In [ ]:
pd.melt(df, id_vars =['state_abbr'], value_vars =[''])

In [7]:
state_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [8]:
df_combined["State"] = df_combined["State"].map(state_codes)

In [11]:
df_combined

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,AL,Baldwin,2010,274,188,80,6,0,0,0,150,71,43,0,0,192,82,0
1,AL,Clay,2010,113,79,34,0,0,0,0,86,60,42,0,0,0,113,0
2,AL,Colbert,2010,355,208,146,1,0,0,0,105,67,47,0,0,114,241,0
3,AL,DeKalb,2010,358,248,106,4,0,0,0,101,74,43,0,0,303,55,0
4,AL,Elmore,2010,241,181,55,5,0,0,0,112,71,43,0,0,241,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,WY,Sublette,2016,366,297,68,1,0,0,0,102,61,44,0,0,351,15,0
1026,WY,Sweetwater,2016,366,304,62,0,0,0,0,100,58,44,0,0,327,4,35
1027,WY,Teton,2016,366,331,35,0,0,0,0,99,50,41,0,0,351,15,0
1028,WY,Uinta,2016,366,341,25,0,0,0,0,87,49,42,0,0,359,0,7


In [14]:
df_combined = df_combined.drop("County", axis="columns")

In [15]:
df_combined.groupby(['Year', 'State']).mean()

Days with AQI  Good Days  Moderate Days  \
Year State                                            
2010 AK            262.12     222.00          35.00   
     AL            282.26     164.89         113.05   
     AR            208.24     138.53          67.35   
     AZ            309.54     199.08         102.23   
     CA            350.40     221.94         102.34   
...                   ...        ...            ...   
2016 VT            301.50     279.75          21.50   
     WA            348.76     317.45          30.34   
     WI            282.93     243.26          35.85   
     WV            272.57     237.79          34.07   
     WY            329.39     286.22          42.94   

            Unhealthy for Sensitive Groups Days  Unhealthy Days  \
Year State                                                        
2010 AK                                    4.12            1.00   
     AL                                    4.00            0.32   
     AR                                    2.24            0.12   
     AZ                                    7.92            0.15   
     CA                                   19.89            5.32   
...                                         ...             ...   
2016 VT                                    0.25            0.00   
     WA                                    0.79            0.17   
     WI                                    3.56            0.26   
     WV                                    0.71            0.00   
     WY                                    0.17            0.06   

            Very Unhealthy Days  Hazardous Days  Max AQI  90th Percentile AQI  \
Year State                                                                      
2010 AK                    0.00            0.00    88.50                44.88   
     AL                    0.00            0.00   112.26                70.32   
     AR                    0.00            0.00   100.00                63.94   
     AZ                    0.00            0.15   233.46                70.38   
     CA                    0.47            0.43   305.60                80.60   
...                         ...             ...      ...                  ...   
2016 VT                    0.00            0.00    83.00                42.75   
     WA                    0.00            0.00    88.17                44.24   
     WI                    0.00            0.00   118.59                57.56   
     WV                    0.00            0.00    95.86                52.57   
     WY                    0.00            0.00    94.78                51.06   

            Median AQI  Days CO  Days NO2  Days Ozone  Days PM2.5  Days PM10  
Year State                                                                    
2010 AK          19.75     2.38      0.00       59.38      191.00       9.38  
     AL          46.16     0.05      0.00      131.05      142.26       8.89  
     AR          43.71     0.00      2.94      110.35       94.94       0.00  
     AZ          42.92     0.00      5.62      191.23       22.77      89.92  
     CA          45.30     0.15      8.91      222.55      105.94      12.85  
...                ...      ...       ...         ...         ...        ...  
2016 VT          29.00     0.00      3.00      217.00       81.50       0.00  
     WA          24.10     0.03      4.03       53.34      287.45       3.90  
     WI          35.96     0.00      1.33      195.41       85.89       0.30  
     WV          35.43     4.93      0.00      160.50       83.21      23.93  
     WY          37.28     0.00      1.78      267.06       16.06      44.50  

[350 rows x 15 columns]